In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11, 4)})
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
'''MAIN DATA'''
data=pd.read_csv('train.csv')
train=data
train=data.drop(['Secondary Sales Amount STC','Secondary Sales Amount Returned','Secondary Sales Amount Invoiced','Secondary Sales Quantity Returned'],axis=1)
train.set_index('DATE',inplace=True)
train.head()


,SKU Code,CFA Code,Channel,Quantity,Secondary Sales Quantity Invoiced,SKU Combination
DATE,,,,,,
2019-01-01,8,0,1,2,0.0,0
2019-02-01,8,0,1,2,0.0,0
2019-03-01,8,0,1,2,0.0,0
2019-04-01,8,0,1,2,192.0,0
2019-05-01,8,0,1,2,864.0,0


In [3]:
'''TEST'''
data_test=pd.read_csv('test.csv')
test=pd.read_csv('test.csv')
test.set_index('DATE',inplace=True)
test.head()

,SKU Code,CFA Code,Channel,Quantity,SKU Combination
DATE,,,,,
2020-01-01,8,0,1,2,0
2020-02-01,8,0,1,2,0
2020-01-01,8,0,2,2,1
2020-02-01,8,0,2,2,1
2020-01-01,8,1,2,2,3


In [4]:
'''PROMOTIONAL DATA'''
p=pd.read_csv('Promotional_Data.csv')

In [5]:
col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('Secondary Sales Quantity Invoiced')

mat_train = np.matrix(train)
mat_test  = np.matrix(test)
mat_new = np.matrix(train.drop('Secondary Sales Quantity Invoiced',axis = 1))
mat_y = np.array(train['Secondary Sales Quantity Invoiced']).reshape(-1,1)

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train,index=data['DATE'])
test  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis,index=data_test['DATE'])

train.head()

,SKU Code,CFA Code,Channel,Quantity,Secondary Sales Quantity Invoiced,SKU Combination
DATE,,,,,,
2019-01-01,0.170213,0.0,0.5,0.2,0.000000,0.0
2019-02-01,0.170213,0.0,0.5,0.2,0.000000,0.0
2019-03-01,0.170213,0.0,0.5,0.2,0.000000,0.0
2019-04-01,0.170213,0.0,0.5,0.2,0.000752,0.0
2019-05-01,0.170213,0.0,0.5,0.2,0.003382,0.0


In [6]:
# List of features
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "Secondary Sales Quantity Invoiced"

# Columns
feature_cols = FEATURES

# Training set and Prediction set with the features to predict
training_set = train[COLUMNS]
prediction_set = train['Secondary Sales Quantity Invoiced']

# Train and Test 
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES] , prediction_set, test_size=0.33, random_state=42)
y_train = pd.DataFrame(y_train, columns = [LABEL])
training_set = pd.DataFrame(x_train, columns = FEATURES).merge(y_train, left_index = True, right_index = True)
training_set.head()

# Training for submission
training_sub = training_set[col_train]

In [7]:
# Same thing but for the test set
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)
testing_set.head()

,SKU Code,CFA Code,Channel,Quantity,SKU Combination,Secondary Sales Quantity Invoiced
DATE,,,,,,
2018-01-01,0.531915,0.333333,0.5,0.4,0.44587,0.000082
2018-01-01,0.531915,0.333333,0.5,0.4,0.44587,0.051217
2018-01-01,0.531915,0.333333,0.5,0.4,0.44587,0.035959
2018-01-01,0.531915,0.333333,0.5,0.4,0.44587,0.012546
2018-01-01,0.531915,0.333333,0.5,0.4,0.44587,0.024411


In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras

seed = 7
np.random.seed(seed)

# Model
model = Sequential()
model.add(Dense(20, input_dim=5, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, kernel_initializer='normal', activation='relu'))
model.add(Dense(2, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
# Compile model
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())

feature_cols = training_set[FEATURES]
labels = training_set[LABEL].values

model.fit(np.array(feature_cols), np.array(labels), epochs=1, batch_size=1)

Epoch 1/1
    9359/18660030 [..............................] - ETA: 32:16:48 - loss: 0.001

KeyboardInterrupt: 